In [9]:
hypo_1 = 'John plays football or chess'
hypo_2 = 'When its raining, John does not play football'
hypo_3 = 'It is raining'

In [10]:
def detect_sentence_structure(hypothesis):
    pass

def or_expression(hypothesis):
   pass 

In [23]:
class Expression:
    def __init__(self, hypothesis):
        
        if hypothesis is None or type(hypothesis) is not str:
            raise ValueError("A hypothesis needs to be of type string and can't be empty.")
        
        self.init_hypo = hypothesis.lower()         # Only use lower case
        self.tokens = self.init_hypo.split(" ")     # Split hypo into tokens
        self.split_references()
        
    def split_references(self):
        for reference in ['or', 'and']:
            if reference in self.tokens:
                reference_idx = self.tokens.index(reference)
                right_tokens = self.tokens[reference_idx + 1:]
                base_tokens = self.tokens[:reference_idx - 1]
                left_tokens = self.tokens[:reference_idx]
                
                self.tokens = left_tokens + [reference] + base_tokens + right_tokens

    def __str__(self):
        return str(self.tokens)

In [24]:
hypo_1.lower()
tokens = hypo_1.split(" ")
"or" in tokens
tokens[tokens.index("or") + 1:]
tokens[:tokens.index("or") - 1]

['John', 'plays']

In [25]:
exp = Expression(hypo_1)
print(exp)

['john', 'plays', 'football', 'or', 'john', 'plays', 'chess']


In [ ]:

class TableauxSolver:
    
    def __init__(self, hypothesis, thesis):
        
        self.hypothesis = hypothesis
        self.thesis = thesis
        
    
